### Aruco sensor processing

Experimentation for the development of an Aruco tag based sensor processing system. 

In [1]:
import sys
sys.path.append("..")
from settings import Config
import numpy as np
import cv2
import torch
import pprint

from sensorprocessing import sp_aruco

# Move data to GPU (if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
from sensorprocessing.aruco.utils import aruco_display

arucoDict = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_5X5_100)
arucoParams = cv2.aruco.DetectorParameters()

In [3]:
# Specify the experiment
experiment = "sensorprocessing_aruco"
run = "aruco_128" 
exp = Config().get_experiment(experiment, run)
pprint.pprint(exp)
sp = sp_aruco.ArucoSensorProcessing(exp, device)

Loading pointer config file: /home/lboloni/.config/BerryPicker/mainsettings.yaml
Loading machine-specific config file: /home/lboloni/Insync/lotzi.boloni@gmail.com/Google Drive/LotziStudy/Code/PackageTracking/BerryPicker/settings/settings-tredy2.yaml
No system dependent experiment file
 /home/lboloni/Insync/lotzi.boloni@gmail.com/Google Drive/LotziStudy/Code/PackageTracking/BerryPicker/settings/experiment-config/Tredy2/sensorprocessing_aruco/aruco_128_sysdep.yaml,
 that is ok, proceeding.
Configuration for experiment: sensorprocessing_aruco/aruco_128 successfully loaded
{'MARKER_COUNT': 10,
 'XMAX': 1000,
 'YMAX': None,
 'data_dir': PosixPath('/home/lboloni/Documents/Hackingwork/__Temporary/BerryPicker-experiments/sensorprocessing_aruco/aruco_128'),
 'exp_run_sys_indep_file': PosixPath('/home/lboloni/Documents/Hackingwork/_Checkouts/BerryPicker/BerryPicker/src/experiment_configs/sensorprocessing_aruco/aruco_128.yaml'),
 'group_name': 'sensorprocessing_aruco',
 'latent_size': 128,
 'run_

In [4]:
camera_number = Config().values["robot"]["active_camera_list"][0]
print(camera_number)
#camera_number = 2
cap = cv2.VideoCapture(camera_number)

while(True):
    ret, frame = cap.read()
    if not ret:
        print("Capture frame returned False, exiting the loop")
        break
    frame = cv2.resize(frame, (640, 480))
    # imgresized = frame

    print(frame)

    z = sp.process(frame)
    print(z)

    # insert ARUCO code here

    marker_corners, marker_ids, rejected_candidates = cv2.aruco.detectMarkers(
        frame, arucoDict, parameters=arucoParams)
    
    detected_markers = aruco_display(marker_corners, marker_ids, rejected_candidates, frame)

    # end of ARUCO code

    # cv2.imshow('Press q to exit', frame)
    cv2.imshow('Press q to exit', detected_markers)
    ret = cv2.waitKey(1)
    if ret != -1:
        ret2 = ret & 0xFF
        if ret2 == ord('q'):
            break
# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()

2
[[[ 69  81  79]
  [ 67  79  77]
  [ 61  80  77]
  ...
  [107 125 119]
  [110 125 121]
  [110 125 121]]

 [[ 62  77  74]
  [ 65  79  77]
  [ 59  80  77]
  ...
  [110 123 117]
  [113 123 121]
  [112 121 120]]

 [[ 60  79  76]
  [ 61  80  77]
  [ 63  85  81]
  ...
  [105 112 106]
  [121 123 116]
  [121 123 116]]

 ...

 [[  2   7   4]
  [  0   2   0]
  [  0   2   0]
  ...
  [ 36  49  45]
  [ 31  45  43]
  [ 32  47  44]]

 [[  0   2   0]
  [  0   2   0]
  [  0   2   0]
  ...
  [ 32  47  44]
  [ 38  45  44]
  [ 39  47  45]]

 [[  0   2   0]
  [  0   2   0]
  [  0   2   0]
  ...
  [ 32  47  44]
  [ 42  45  44]
  [ 43  46  45]]]
(480, 640, 3)


AttributeError: 'numpy.ndarray' object has no attribute 'permute'

In [ ]:
print(marker_corners)

(array([[[587., 137.],
        [608., 136.],
        [608., 154.],
        [587., 155.]]], dtype=float32), array([[[531., 203.],
        [550., 203.],
        [550., 221.],
        [531., 221.]]], dtype=float32), array([[[533., 141.],
        [550., 139.],
        [550., 156.],
        [533., 158.]]], dtype=float32))


In [ ]:
marker_ids

array([[2],
       [9],
       [1]], dtype=int32)

In [ ]:
frame.shape

(480, 640, 3)

This code creates the encoding for the Aruco markers because 

In [ ]:
MARKER_COUNT = 10
XMAX = float(frame.shape[1])
YMAX = float(frame.shape[0])
NORMALIZER = np.tile([XMAX, YMAX], 4)

z = np.ones(MARKER_COUNT * (8 + 1)) * -1.0
for id, corners in zip(marker_ids, marker_corners):
    detection = corners[0].flatten() / NORMALIZER
    idn = id[0]
    z[idn * (8+1):(idn+1) * (8+1)-1] = detection


In [ ]:
z[0:9]

array([-1., -1., -1., -1., -1., -1., -1., -1., -1.])